# Sensitivity analysis of the NF-kB pathway
The steps for this are as follow:
1. From an equations text file, load the reaction equations and create a state space model
2. From the state space model and initial parameters, find 
    1. The initial parameters for the model (run it 1000 times from the first initial conditions)
    2. Determine the value of x<sub>i</sub> for all i for all t in [0, N]
3. Calculate s<sub>i, j</sub> for all t given the values of x<sub>i</sub> for all t
4. Calcuate sbar for all t
5. Having the sensitivity matrix and the values of x for all desired t, calculate the sensitivity equations

We need x<sub>i</sub>(t) and s<sub>i, j</sub> to calculate sbar, which is needed for sensitivity analysis. Sensitivity analysis for all constants is defined as 
![Equation 22](./equation22.png)
where s bar is
![s bar](./sbar.png)
dx/dtheta can be found from S
![S](./bigS.png)
and an entry  s<sub>i, j</sub> is 
![s](./littleS.png)
where x<sub>i</sub> is the integration of the system of equations from the state space model and delta is the Kronecker delta function 
![kronecker delta](./kroneckerDelta.png)
### All necessary imports

In [2]:
import copy
import numpy as np
import sympy as sp
import math
from scipy.integrate import solve_ivp
import os,glob

### All program constants

In [3]:
initial_conditions_N = 2000
N = 400

pathway_file = 'pathway_eqs.txt'
reaction_coefficients_file = 'reaction_coefficients.txt'

### Load the constants and separate them 

In [4]:
ks = {}
with open(reaction_coefficients_file, 'r') as o: 
    for l in o:
        k = l.split('=')[0]
        ks[k] = float(eval(l.split('=')[1].replace('\n', '').strip()))

In [5]:
ks

{'k1': 0.5,
 'k2': 0.0005,
 'k3': 0.5,
 'k4': 0.0005,
 'k5': 0.5,
 'k6': 0.0005,
 'k7': 0.5,
 'k8': 0.0005,
 'k9': 0.0204,
 'k10': 0.5,
 'k11': 0.0005,
 'k12': 0.0075,
 'k13': 0.5,
 'k14': 0.0005,
 'k15': 0.011000000000000001,
 'k16': 2.25e-05,
 'k17': 2.25e-05,
 'k18': 2.25e-05,
 'k19': 0.09000000000000001,
 'k20': 8e-05,
 'k21': 0.5,
 'k22': 0.0005,
 'k23': 0.5,
 'k24': 0.0005,
 'k25': 0.5,
 'k26': 0.0005,
 'k27': 1.5399999999999999e-06,
 'k28': 0.0165,
 'k29': 0.00028,
 'k30': 1.78e-07,
 'k31': 0.00028,
 'k32': 1.2699999999999999e-07,
 'k33': 0.00028,
 'k34': 0.0225,
 'k35': 0.00125,
 'k36': 0.00408,
 'k37': 0.000113,
 'k38': 0.00030000000000000003,
 'k39': 0.0002,
 'k40': 0.006,
 'k41': 0.00175,
 'k42': 0.00408,
 'k43': 0.000113,
 'k44': 0.00015000000000000001,
 'k45': 0.0001,
 'k46': 0.009000000000000001,
 'k47': 0.00175,
 'k48': 0.00408,
 'k49': 0.000113,
 'k50': 0.00015000000000000001,
 'k51': 0.0001,
 'k52': 0.18500000000000003,
 'k53': 0.00125,
 'k54': 0.0138,
 'k55': 0.048,
 

## 1. Generate the state space model from reaction equations

In [6]:
import numpy as np

file = open(pathway_file,"r")
eqs_arr=file.readlines()
file.close()
eqs_arr=[i[:-1] if i[-1] == '\n' else i for i in eqs_arr ] #remove \n

class Rxn:
    
    def __init__(self, ID, reactants, products):
        #reactants and products are lists
        self.ID = str(ID)
        self.reactants = reactants
        self.products = products
        
    def __str__(self):
        ret = "ID: " + self.ID + " Reactants: "
        for r in self.reactants:
            ret = ret + r.ID + ", "
        ret = ret + "Products: "
        for p in self.products:
            ret = ret + p.ID + ", "
        return ret

class RxnSpec:
    
    def __init__(self, ID, name):
        self.ID = 'x' + str(ID)
        self.name = name
        
    def __str__(self):
        return 'ID: ' + self.ID + ' name: ' + self.name

#list of rxn species
rxn_species = [RxnSpec(1,'ikba'), RxnSpec(2,'nf-kb'), RxnSpec(3,'ikba-nf-kb'), RxnSpec(4,'ikbb'), RxnSpec(5,'ikbb-nf-kb'),
              RxnSpec(6,'ikbe'),RxnSpec(7,'ikbe-nf-kb'),RxnSpec(8,'ikkikba'),RxnSpec(9,'ikkikba-nf-kb'),RxnSpec(10,'ikk'),
              RxnSpec(11,'ikkikbb'),RxnSpec(12,'ikkikbb-nf-kb'),RxnSpec(13,'ikkikbe'),RxnSpec(14,'ikkikbe-nf-kb'),
              RxnSpec(15,'nf-kb_n'),RxnSpec(16,'ikba_n'),RxnSpec(17,'ikba_n-nf-kb_n'),RxnSpec(18,'ikbb_n'),RxnSpec(19,'ikbb_n-nf-kb_n'),
              RxnSpec(20,'ikbe_n'),RxnSpec(21,'ikbe_n-nf-kb_n'),RxnSpec(22,'ikba_-t'),RxnSpec(23,'ikbb_-t'),RxnSpec(24,'ikbe_-t'),RxnSpec('SOURCE','source'),RxnSpec('SINK','sink')]

rxn_arr = []

for idx,eq in enumerate(eqs_arr):
    temp = eq.split('->')
    reactants = temp[0].split('+')
    products = temp[1].split('+')
    #convert each element in reactants and products into a RxnSpec
    new_reactants = []
    new_products = []
    for reac in reactants:
        for rs in rxn_species:
            if rs.name == reac:
                new_reactants.append(rs)
                break
    for prod in products:
        for rs in rxn_species:
            
            if rs.name == prod:
                new_products.append(rs)
                break
    
    
    rxn = Rxn(idx+1,new_reactants, new_products)
    rxn_arr.append(rxn)

state_space_model = []

for rs in rxn_species:
    eq = rs.ID + '(t) = '
    for rxn in rxn_arr:
        if rs.name in [i.name for i in rxn.reactants]:
            k = '-k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
            
        if rs.name in [i.name for i in rxn.products]:
            k = '+k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
    state_space_model.append(eq)
    
for eq in state_space_model:
    print(eq)

x1(t) = -k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
x2(t) = -k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15
x3(t) = +k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17
x4(t) = -k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18
x5(t) = +k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19
x6(t) = -k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20
x7(t) = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
x8(t) = -k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8
x9(t) = +k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9
x10(t) = +k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13
x11(t) = -k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11
x12(t) = +k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12
x13(t) = -k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13
x14(

## 2. Identify the initial conditions and x(t) for all t
1. Run the integration initial_conditions_N number of times to get initial conditions
2. Augment these by any thing you need from the paper for actual initial conditions

__NOTE__:   as of now, hard code the k values (from some source) and the partial derivatives (from the state space model)

In [7]:
# followed the direction of this video https://www.youtube.com/watch?v=zRMmiBMjP9o
def nf_kb(t, x):
    # x is here to be used by (1) initial conditions and (2) x when we use odeint
    # t is here for obvious reasons
    
    # define the k values
    k1=0.5
    k2=0.5*10**-3 
    k3=0.5
    k4=0.5*10**-3 
    k5=0.5
    k6=0.5*10**-3 
    k7=0.5
    k8=0.5*10**-3 
    k9=2.04*10**-2 
    k10=0.5
    k11=0.5*10**-3 
    k12=7.5*10**-3 
    k13=0.5
    k14=0.5*10**-3 
    k15=1.1*10**-2 
    k16=2.25*10**-5 
    k17=2.25*10**-5 
    k18=2.25*10**-5 
    k19=0.9*10**-1 
    k20=0.8*10**-4 
    k21=0.5
    k22=0.5*10**-3 
    k23=0.5
    k24=0.5*10**-3 
    k25=0.5
    k26=0.5*10**-3 
    k27=1.54*10**-6  
    k28=1.65*10**-2  
    k29=2.8*10**-4 
    k30=1.78*10**-7  
    k31=2.8*10**-4 
    k32=1.27*10**-7  
    k33=2.8*10**-4 
    k34=22.5*10**-3  
    k35=1.25*10**-3 
    k36=4.08*10**-3 
    k37=1.13*10**-4 
    k38=3*10**-4 
    k39=2*10**-4 
    k40=6.0*10**-3  
    k41=1.75*10**-3 
    k42=4.08*10**-3 
    k43=1.13*10**-4 
    k44=1.5*10**-4 
    k45=1*10**-4 
    k46=9.0*10**-3  
    k47=1.75*10**-3 
    k48=4.08*10**-3 
    k49=1.13*10**-4 
    k50=1.5*10**-4 
    k51=1*10**-4 
    k52=1.85*10**-1  
    k53=1.25*10**-3 
    k54=1.38*10**-2 
    k55=4.8*10**-2  
    k56=1.75*10**-3 
    k57=5.2*10**-3 
    k58=7.0*10**-2  
    k59=1.75*10**-3 
    k60=5.2*10**-3 
    k61=1.2*10**-4 
    k62=4.07*10**-3 
    k63=1.5*10**-3 
    k64=2.2*10**-3 
    
    # im lazy and make it easy to read
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]
    x5 = x[4]
    x6 = x[5]
    x7 = x[6]
    x8 = x[7]
    x9 = x[8]
    x10 = x[9]
    x11 = x[10]
    x12 = x[11]
    x13 = x[12]
    x14 = x[13]
    x15 = x[14]
    x16 = x[15]
    x17 = x[16]
    x18 = x[17]
    x19 = x[18]
    x20 = x[19]
    x21 = x[20]
    x22 = x[21]
    x23 = x[22]
    x24 = x[23]
    
    dx1dt = -k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
    dx2dt = -k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15
    dx3dt = +k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17
    dx4dt = -k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18
    dx5dt = +k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19
    dx6dt = -k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20
    dx7dt = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
    dx8dt = -k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8
    dx9dt = +k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9
    dx10dt = +k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13
    dx11dt = -k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11
    dx12dt = +k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12
    dx13dt = -k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13
    dx14dt = +k13*x13*x2-k14*x14-k15*x14+k58*x10*x7-k59*x14
    dx15dt = +k19*x2-k20*x15-k21*x16*x15+k22*x17-k23*x18*x15+k24*x19-k25*x20*x15+k26*x21-k28*x15*x15+k28*x15*x15
    dx16dt = -k21*x16*x15+k22*x17+k38*x1-k39*x16
    dx17dt = +k21*x16*x15-k22*x17-k54*x17
    dx18dt = -k23*x18*x15+k24*x19+k44*x4-k45*x18
    dx19dt = +k23*x18*x15-k24*x19-k57*x19
    dx20dt = -k25*x20*x15+k26*x21+k50*x6-k51*x20
    dx21dt = +k25*x20*x15-k26*x21-k60*x21
    dx22dt = +k27+k28*x15*x15-k29*x22-k36*x22+k36*x22
    dx23dt = +k30-k31*x23-k42*x23+k42*x23
    dx24dt = +k32-k33*x24-k48*x24+k48*x24
    
    
    return [dx1dt, dx2dt, dx3dt, dx4dt, dx5dt, dx6dt, dx7dt, dx8dt,
            dx9dt, dx10dt, dx11dt, dx12dt, dx13dt, dx14dt, dx15dt, dx16dt, 
            dx17dt, dx18dt, dx19dt, dx20dt, dx21dt, dx22dt, dx23dt, dx24dt]
# first run for the initial conditions
T = initial_conditions_N 
x0x0 = [0, 0.1] + [0 for i in range(22)]
t = [i for i in range(T+1)]
init_conditions = solve_ivp(nf_kb, (min(t), max(t)), x0x0, t_eval=t).y[:, -1] # only want the last values


Now we have the initial conditions, so we need to augment this by paper specific parameters

In [8]:
# ok now do it for the simulation
T = N + 1
x0 = init_conditions
# fill in the extra stuff in the paper, IKK at 0.1 (x10, index 9) PAPER SPECIFIC
x0[9] = 0.1
t = [i for i in range(T)]

x = solve_ivp(nf_kb, (min(t), max(t)), x0, t_eval=t).y
# x has the shape of 
# [N, i] (an entry for all x for all N)
# replace x[0] with the initial conditions
x = x[:, 1:]
for row in x:
    print(row)

[0.59668724 0.59564677 0.59463354 0.59364461 0.5926754  0.59172024
 0.59077887 0.58985129 0.58893665 0.58803361 0.58714134 0.58626006
 0.58538981 0.5845304  0.58368141 0.5828422  0.58201222 0.58119175
 0.58038083 0.57957941 0.57878737 0.57800452 0.57723059 0.57646524
 0.57570807 0.57495819 0.57421516 0.57347984 0.57275287 0.57203467
 0.57132543 0.57062513 0.56993348 0.56924999 0.56857393 0.56790436
 0.56724008 0.56657968 0.56592261 0.56527191 0.56462885 0.56399387
 0.56336668 0.56274632 0.56213112 0.56151949 0.56091309 0.56031274
 0.55971873 0.55913105 0.55854934 0.55797295 0.5574009  0.55683243
 0.55626837 0.5557095  0.55515631 0.55460895 0.55406729 0.55353088
 0.55299898 0.55247052 0.55194428 0.55142098 0.55090204 0.55038834
 0.54988029 0.5493778  0.54888031 0.54838681 0.54789578 0.54740556
 0.54691737 0.54643311 0.54595396 0.5454805  0.54501264 0.54454968
 0.54409029 0.54363248 0.54317409 0.54271732 0.54226432 0.54181607
 0.54137287 0.54093441 0.5404997  0.54006714 0.53963469 0.5392

In [13]:
len(x0)

24

## 3. Calculate s for all t
### Finding dx with respect to theta
Here are the equations

![dx dtheat](dxdtheta.png)

Once we can get equation 9, we can integrate with respect to time to get dx dtheta.

s i j is defined as 

![s](littleS.png)

I think f are the system of equations from the state space model

I'm using the variable `p` and `p_i` to fill in for dx/dtheta

In [9]:
# here is the state space model

ssm = []
ssm.append(sp.sympify('-k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16'))
ssm.append(sp.sympify('-k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15'))
ssm.append(sp.sympify('+k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17'))
ssm.append(sp.sympify('-k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18'))
ssm.append(sp.sympify('+k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19'))
ssm.append(sp.sympify('-k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20'))
ssm.append(sp.sympify('+k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21'))
ssm.append(sp.sympify('-k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8'))
ssm.append(sp.sympify('+k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9'))
ssm.append(sp.sympify('+k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13'))
ssm.append(sp.sympify('-k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11'))
ssm.append(sp.sympify('+k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12'))
ssm.append(sp.sympify('-k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13'))
ssm.append(sp.sympify('+k13*x13*x2-k14*x14-k15*x14+k58*x10*x7-k59*x14'))
ssm.append(sp.sympify('+k19*x2-k20*x15-k21*x16*x15+k22*x17-k23*x18*x15+k24*x19-k25*x20*x15+k26*x21-k28*x15*x15+k28*x15*x15'))
ssm.append(sp.sympify('-k21*x16*x15+k22*x17+k38*x1-k39*x16'))
ssm.append(sp.sympify('+k21*x16*x15-k22*x17-k54*x17'))
ssm.append(sp.sympify('-k23*x18*x15+k24*x19+k44*x4-k45*x18'))
ssm.append(sp.sympify('+k23*x18*x15-k24*x19-k57*x19'))
ssm.append(sp.sympify('-k25*x20*x15+k26*x21+k50*x6-k51*x20'))
ssm.append(sp.sympify('+k25*x20*x15-k26*x21-k60*x21'))
ssm.append(sp.sympify('+k27+k28*x15*x15-k29*x22-k36*x22+k36*x22'))
ssm.append(sp.sympify('+k30-k31*x23-k42*x23+k42*x23'))
ssm.append(sp.sympify('+k32-k33*x24-k48*x24+k48*x24'))

In [10]:
# get Sj. ONLY FOR INITIAL CONDITIONS OF SOLVING THE NEXT SYSTEM OF EQUATIONS
def kdf(i, j):
    if i != j:
        return 0
    return ks['k{}'.format(j+1)] - x0[i]

def sj(j):
    return [kdf(i, j) for i in range(len(x0))]

In [11]:
# get Fj (df/dthetaj)
def fj(j):
    return [sp.diff(ssm[i], 'k{}'.format(j+1)) for i in range(len(x0))]

In [12]:
# find J
def makeJ():
    J = []
    # df1 is top row where as dx1 is the first column
    # is dfn/dfn
    for i in range(len(ssm)):
        row = []
        tod = ssm[i]
        for n in range(len(ssm)):
            respectto = 'x{}'.format(n+1)
            row.append(sp.diff(tod, respectto))
        J.append(row)
    return J

In [13]:
J = makeJ()
def ddtdxdthetaj(j, p):
    return np.add(np.dot(np.asarray(J), p), np.asarray(fj(j)))

In [14]:
# linear interpolation for x 
# x is i by N shape, so index i then t
thresh = .000001
def interpx(i, t):
    if abs(t - int(t)) < thresh:
        return x[i][int(t)]
    t1 = int(math.floor(t))
    t2 = int(math.ceil(t))
    x1 = x[i][t1]
    x2 = x[i][t2]
    return ((t-t1) * (x2 - x1)/(t2-t1)) + x1

In [15]:
len(ks)

64

In [16]:
def integratethis(t, p, soe):
    tosub = {}
    tosolve = copy.deepcopy(soe)
    for i in range(len(x0)):
        tosub['x{}'.format(i + 1)] = interpx(i, t)
    for k in ks:
        tosub[k] = ks[k]
    for pi in range(len(p)):
        tosub['p{}'.format(pi+1)] = p[pi]
    for j in range(len(tosolve)):
        tosolve[j] = float(tosolve[j].evalf(subs=tosub))
    return list(tosolve)

N = 400
t = [i for i in range(N)]
# the form of this solved system will be (j, N, i) so first index is the theta, the second is the time, and the third is the x
solveddxdtheta = []
for j in range(len(ks)): 
    print('solving dx dtheta for j {}/{}\r'.format(j+1, len(ks)), end='')
    soe = ddtdxdthetaj(j, [sp.sympify('p{}'.format(i + 1)) for i in range(len(x0))])
    p0 = sj(j)
    solveddxdtheta.append(solve_ivp(integratethis, (min(t), max(t)), p0, args=(soe, ), t_eval=t).y)
    

C:\Users\Suchita Lulla\Anaconda3\lib\site-packages\scipy\integrate\_ivp\common.py:32: UserWarning: The following arguments have no effect for a chosen solver: `args`.
  .format(", ".join("`{}`".format(x) for x in extraneous)))


TypeError: integratethis() missing 1 required positional argument: 'soe'

In [26]:
#loading in dsolveddxdtheta from files
solveddxdtheta = []
folder_path = "nfkb_out"
extension = "*"
for filename in glob.glob(os.path.join(folder_path, extension)):
    param_ls = []
    with open(filename, 'r') as f:
            lines = f.readlines()[2:] #remove "you've requested..."
            #print(lines[0])
            ls_strings = []
            stri = []
            for line in lines:
                if "]" in line:
                    stri += line[:line.index("]")+1]
                    ls_strings.append(stri)
                if "[" in line:
                    stri = line[line.index("[")]
                
                
                else:
                    stri += line
            #print(ls_strings[0])
                
            #now convert to a list of ints
            ls_nums = []
            for stri in ls_strings:
                stri = stri[1:-1] #remove brackets
                stri = stri.split(" ")#split into individual numbers
                stri = [s.rstrip() for s in stri if s != ""] #remove newlines, exclude spaces
                stri = [float(s) for s in stri]
                ls_nums.append(stri)
            solveddxdtheta.append(ls_nums)
                
            
                    

In [28]:
len(solveddxdtheta)

64

In [34]:
len(solveddxdtheta[0])

24

In [60]:
print(solveddxdtheta[10][0])

[-1.39348147e-06, -2.24195665e-06, -3.32870026e-06, -4.62163908e-06, -6.11823683e-06, -7.8153956e-06, -9.70819625e-06, -1.17898984e-05, -1.40519406e-05, -1.64839401e-05, -1.90736928e-05, -2.18071736e-05, -2.46463421e-05, -2.74942867e-05, -3.04329137e-05, -3.35436181e-05, -3.68785055e-05, -4.04603927e-05, -4.42828074e-05, -4.83099884e-05, -5.24768855e-05, -5.66891595e-05, -6.0823182e-05, -6.47210719e-05, -6.83263074e-05, -7.1955516e-05, -7.58781632e-05, -8.02480002e-05, -8.51030635e-05, -9.03656755e-05, -9.58424439e-05, -0.000101224262, -0.000106086309, -0.000110180375, -0.000114117288, -0.00011833098, -0.000123068382, -0.000128398841, -0.000134214115, -0.000140228377, -0.000145978214, -0.000150866455, -0.000155193572, -0.000159591112, -0.00016439651, -0.000169769138, -0.000175690308, -0.000181963267, -0.000188213202, -0.000193887237, -0.000198626032, -0.000203132278, -0.000207881014, -0.000213148164, -0.000219024108, -0.000225413679, -0.000232036164, -0.000238425305, -0.00024393471, -0

## 4. S bar
s bar is defined as:
![s bar](./sbar.png)

In [95]:
# find sbar for all t
def sbar(i, j, k):
    #return solveddxdtheta[j][k][i] * ks['k{}'.format(j+1)] / x[i][k]
    return solveddxdtheta[j][i][k] * ks['k{}'.format(j+1)] / x[i][k]

## 5 the sensitivity constants 

### Dynamic sensitivity analysis

#### Dynamic sensitivity analysis with multiple variables
This is equation 22 of the paper
![Equation 22](./equation22.png)

In [96]:
# do the innermost summation
import math
def inner_sum(k, j, i_range):
    tosum = []
    for i in range(i_range):
        added = sbar(i, j, k)
        tosum.append(sbar(i, j, k) ** 2)
    return sum(tosum)

In [97]:
# do the outer summation
def outer_sum(N, j, i_range):
    tosum = []
    for k in range(N):
        tosum.append(inner_sum(k, j, i_range))
    return sum(tosum)

In [98]:
# do the 1/n and the sqrt
def eqn22(N, j, i_range):
    return (1/N) * np.sqrt(outer_sum(N, j, i_range))

In [99]:
# do the sensitivity analysis
i_range = len(x0)
j_range = len(ks.keys())
N=376 #changed this ***
OS = []
for j in range(j_range):
    print('aaa')
    print('Calculating sensitivity for j {}/{}\r'.format(j + 1, j_range), end='')
    OS.append(eqn22(N, j, i_range))
    

aaa
aaaculating sensitivity for j 1/64
aaaculating sensitivity for j 2/64
aaaculating sensitivity for j 3/64
aaaculating sensitivity for j 4/64
aaaculating sensitivity for j 5/64
aaaculating sensitivity for j 6/64
aaaculating sensitivity for j 7/64
aaaculating sensitivity for j 8/64
aaaculating sensitivity for j 9/64
aaaculating sensitivity for j 10/64
aaaculating sensitivity for j 11/64
aaaculating sensitivity for j 12/64
aaaculating sensitivity for j 13/64
aaaculating sensitivity for j 14/64
aaaculating sensitivity for j 15/64
aaaculating sensitivity for j 16/64
aaaculating sensitivity for j 17/64
aaaculating sensitivity for j 18/64
aaaculating sensitivity for j 19/64
aaaculating sensitivity for j 20/64
aaaculating sensitivity for j 21/64
aaaculating sensitivity for j 22/64
aaaculating sensitivity for j 23/64
aaaculating sensitivity for j 24/64
aaaculating sensitivity for j 25/64
aaaculating sensitivity for j 26/64
aaaculating sensitivity for j 27/64
aaaculating sensitivity for j 28/

In [100]:
print(OS)

[0.06872004495484937, 0.009572494995791655, 0.2877254589852152, 0.003840516878665603, 135.93480716743593, 0.003748353167132455, 184.0874754363814, 0.0018539147587484904, 0.07119211477811892, 0.3732577738453145, 0.002773727369606119, 0.5062445229784241, 1350.6265269467647, 0.0027256953921574916, 0.03149975167829393, 0.00048799940572562045, 0.0001818172097541903, 0.00017814442702756054, 24.92357041260492, 0.00017427947839254016, 1078.2570374397717, 0.0017791719776740025, 43.82982314457524, 0.0020744766894439434, 0.03401883642910207, 0.0020792038206442934, 0.0001598257689426581, 2.060656746778229e-05, 0.003274810271867302, 0.006609512045039552, 0.0029918889857109094, 0.006615505191526529, 0.0029945074597832865, 0.07403652354959697, 0.009694096169815767, 0.022548585656262176, 0.0029385060275874206, 0.03600978136873405, 0.0026295700154369303, 0.05148981784892505, 0.009215619413310096, 0.01424792488193552, 0.0014124330811545634, 0.027831287947053945, 0.0009132515024229123, 0.0512185760556000

In [101]:
np.argmax(OS)

12

In [102]:
counted = [(i, OS[i]) for i in range(len(OS))]

In [103]:
counted.sort(key=lambda x: x[1], reverse=True)

In [105]:
print(counted)

[(12, 1350.6265269467647), (20, 1078.2570374397717), (6, 184.0874754363814), (4, 135.93480716743593), (22, 43.82982314457524), (18, 24.92357041260492), (11, 0.5062445229784241), (9, 0.3732577738453145), (2, 0.2877254589852152), (51, 0.0825589023306935), (33, 0.07403652354959697), (8, 0.07119211477811892), (0, 0.06872004495484937), (39, 0.05148981784892505), (45, 0.051218576055600054), (54, 0.04556051502410932), (57, 0.04341163275582534), (53, 0.037745652738932885), (61, 0.03737533128881624), (37, 0.03600978136873405), (24, 0.03401883642910207), (14, 0.03149975167829393), (43, 0.027831287947053945), (49, 0.02764611499357248), (59, 0.024914619487134717), (56, 0.0248638834549812), (35, 0.022548585656262176), (47, 0.014630086160566163), (41, 0.01424792488193552), (63, 0.011454916128795086), (34, 0.009694096169815767), (1, 0.009572494995791655), (40, 0.009215619413310096), (46, 0.008841838476722213), (62, 0.008056955473572986), (31, 0.006615505191526529), (29, 0.006609512045039552), (55, 0.